## 1 — LLM Chat Structure (Conceptual)

LLM chat systems use a list of messages with roles: `system`, `user`, and `assistant`. The `system` message sets behavior, `user` provides instructions or queries, and `assistant` contains model outputs. We'll demonstrate how this structure maps to function calls and simple agent loops.

In [ ]:
# Example: message structure example (no external API)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize the dataset columns for me."},
]
print("Messages structure:
", messages)

: 

## 2 — Prompt Design (Short)
Prompts guide model behavior. Start with a clear `system` instruction, then provide context and ask concise questions. Use few-shot examples for structure when needed.

In [ ]:
# Prompt template example
from string import Template
template = Template("System: $system
User: $user_prompt")
print(template.substitute(system="You summarize tables", user_prompt="Describe columns"))

## 3 — Memory (Conceptual)
Memory stores the conversation history or facts about the user. Keep memory small, summarize long histories, and avoid storing secrets like API keys.

In [ ]:
# Simple in-memory conversation store example
class Memory:
    def __init__(self):
        self.history = []
    def add(self, role, content):
        self.history.append({"role": role, "content": content})
    def summarize(self):
        return ' | '.join(m['content'] for m in self.history[-5:])

mem = Memory()
mem.add('user', 'I like data visualization')
mem.add('assistant', 'Noted — I will suggest charts')
print(mem.summarize())

## Exercises
- Modify the `Memory` class to limit storage by total characters rather than message count.
- Write a function that converts the memory into a system prompt for future LLM calls.

---
End of `01_basics_overview.ipynb`. Continue with `02_code_examples.ipynb` for runnable code examples calling local dummy tools and structured explanations.